<a href="https://colab.research.google.com/github/Kaveesha20/MYSLT-app-Feature-Request/blob/developer/feature_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Scraping MYSLT App Reviews

In [1]:
!pip install google-play-scraper pandas  # Installs Python packages,scrape Google Play Store reviews,DataFrames
from google_play_scraper import reviews_all, Sort #fetch all available reviews,sort reviews
import pandas as pd

# Fetch MYSLT App Reviews
reviews = reviews_all(
    'com.slt.selfcare',  # MYSLT App ID
    lang='en',# Only English reviews
    country='lk', # Sri Lanka
    sort=Sort.NEWEST,# Get newest reviews first
    sleep_milliseconds=1000, # Delay to avoid being blocked
)
# Convert to DataFrame and rename 'at' to 'date'
df = pd.DataFrame(reviews)
if 'at' in df.columns:
    df.rename(columns={'at': 'date'}, inplace=True)
df['source'] = 'Google Play'

# Save to CSV
df = pd.DataFrame(reviews)
df.to_csv('myslt_reviews.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('myslt_reviews.csv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:

# Print the extracted reviews
for review in reviews:
    print(f"User: {review['userName']}")
    print(f"Rating: {review['score']}")
    print(f"Review: {review['content']}")
    print("-" * 20)

User: Tharusha Lakshan
Rating: 1
Review: Bill pay option is not working.it says it will available soon since about 1 year.
--------------------
User: Pasan Ayeshmantha
Rating: 1
Review: useless full of errors
--------------------
User: hiran Ihaladeniya
Rating: 1
Review: App features won't work properly and some bugs did not fix a long time
--------------------
User: Lahiru Prasanna (Sl FireFlash)
Rating: 3
Review: There is so much important work that can be done to UIs. Core functionality seems okay so far.
--------------------
User: Yohan Chanuka
Rating: 1
Review: The worst app u can find in the store.
--------------------
User: gihan abeykoon
Rating: 1
Review: pathatic
--------------------
User: tharindu Randunu
Rating: 1
Review: service temporary unavailable please try again later !!!!! 😂😂😂
--------------------
User: Eranga Priyadarshana
Rating: 1
Review: poor
--------------------
User: Rodger Jay
Rating: 1
Review: useless app, full of bugs
--------------------
User: Ruchira Kotuwe

In [3]:
!pip install app_store_scraper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requ

In [4]:
import requests
import pandas as pd

def fetch_app_store_reviews(app_id="1492064957", pages=2):
    reviews = []
    for page in range(1, pages+1):
        url = f"https://itunes.apple.com/rss/customerreviews/page={page}/id={app_id}/sortby=mostrecent/json"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue
        data = response.json()
        entries = data.get('feed', {}).get('entry', [])[1:]  # Skip app metadata
        for entry in entries:
            reviews.append({
                'userName': entry['author']['name']['label'],
                'content': entry['content']['label'],
                'score': entry['im:rating']['label'],
                'date': entry['updated']['label'],
                'source': 'App Store'
            })
    return reviews

# Fetch and save reviews
app_reviews = fetch_app_store_reviews()
df = pd.DataFrame(app_reviews)
df.to_csv('myslt_appstore_reviews.csv', index=False)

# Download the CSV file
files.download('myslt_appstore_reviews.csv')

# Print sample reviews
for review in app_reviews[:5]:
    print(f"User: {review['userName']}")
    print(f"Rating: {review['score']}")
    print(f"Review: {review['content']}")
    print("-" * 20)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

User: HemanthaJ
Rating: 1
Review: I don’t know why this developer doesn’t look other competitors application.Functionals are very poor and it makes frustrate the users.
--------------------
User: Mali(&(&(&(
Rating: 1
Review: Bill outstanding is not updating even though I paid the full bill amount
--------------------
User: Mali(&(&(&(
Rating: 1
Review: Total Payable is not even updating accurately eventhough i fully settled the monthly bill. mybill section in slt.lk is better than this app
--------------------
User: Susa.abey
Rating: 1
Review: A useless app. Never works.
--------------------
User: RareLime.exe
Rating: 1
Review: Yes. It’s not functioning properly
--------------------


# Detect and Translate Sinhala Reviews

In [5]:
!pip install langdetect deep-translator pandas

import pandas as pd
from langdetect import detect
from deep_translator import GoogleTranslator
import logging
from google.colab import files

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize translator
translator = GoogleTranslator(source='auto', target='en')

# Translation function
def translate_review(text):
    try:
        if not text or pd.isna(text):
            return text
        if detect(text) != 'en':
            translated = translator.translate(text)
            logging.info(f"Translated: {text} -> {translated}")
            return translated
        return text
    except Exception as e:
        logging.warning(f"Translation failed for text: {text}. Error: {e}")
        return text

# Preprocess reviews
def preprocess_reviews(google_csv, appstore_csv):
    try:
        # Load Google Play reviews
        df_google = pd.read_csv(google_csv)
        if 'at' in df_google.columns:
            df_google.rename(columns={'at': 'date'}, inplace=True)
        df_google['source'] = 'Google Play'
        df_google = df_google[['userName', 'content', 'score', 'date', 'source']].dropna(subset=['content'])

        # Load App Store reviews
        df_appstore = pd.read_csv(appstore_csv)
        df_appstore = df_appstore[['userName', 'content', 'score', 'date', 'source']].dropna(subset=['content'])

        # Combine reviews
        df = pd.concat([df_google, df_appstore], ignore_index=True)
        logging.info(f"Loaded {len(df)} reviews from both stores")

        # Apply translation
        df['translated_content'] = df['content'].apply(translate_review)

        # Parse dates
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['year'] = df['date'].dt.year

        # Save preprocessed reviews
        df.to_csv('myslt_preprocessed_reviews.csv', index=False)
        files.download('myslt_preprocessed_reviews.csv')
        logging.info("Saved preprocessed reviews to myslt_preprocessed_reviews.csv")

        return df

    except Exception as e:
        logging.error(f"Error preprocessing reviews: {e}")
        raise

# Example usage
if __name__ == "__main__":
    try:
        df = preprocess_reviews('myslt_reviews.csv', 'myslt_appstore_reviews.csv')
        print("Preprocessing complete. Downloaded: myslt_preprocessed_reviews.csv")
        print("\nSample of preprocessed reviews:")
        print(df[['userName', 'content', 'translated_content', 'source']].head())
    except Exception as e:
        print(f"Failed to preprocess reviews: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ab5224246d9889ecd89287b81d8fa9b996b3ff81e363ed0e91b169ed94ebebce
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Preprocessing complete. Downloaded: myslt_preprocessed_reviews.csv

Sample of preprocessed reviews:
                         userName  \
0                Tharusha Lakshan   
1               Pasan Ayeshmantha   
2               hiran Ihaladeniya   
3  Lahiru Prasanna (Sl FireFlash)   
4                   Yohan Chanuka   

                                             content  \
0  Bill pay option is not working.it says it will...   
1                             useless full of errors   
2  App features won't work properly and some bugs...   
3  There is so much important work that can be do...   
4             The worst app u can find in the store.   

                                  translated_content       source  
0  Bill pay option is not working.it says it will...  Google Play  
1                             useless full of errors  Google Play  
2  App features won't work properly and some bugs...  Google Play  
3  There is so much important work that can be do...  Google Play  


In [28]:
!pip install transformers pandas

from transformers import pipeline
import pandas as pd
import logging
from google.colab import files

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load model
try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)
    logging.info("Model loaded successfully")
except Exception as e:
    logging.error(f"Failed to load model: {e}")
    raise

# Refined categories for MYSLT
main_categories = [
    "UI/UX Enhancement",
    "Payment Features",
    "Authentication",
    "Language Support",
    "Notifications",
    "Data Usage Tracking",
    "Customer Support",
    "Other"
]

category_mapping = {
    "UI/UX Enhancement": "UI/UX",
    "Payment Features": "Payment",
    "Authentication": "Login",
    "Language Support": "Language",
    "Notifications": "Notifications",
    "Data Usage Tracking": "Data Usage",
    "Customer Support": "Support",
    "Other": "Other"
}

sub_categories = {
    "UI/UX Enhancement": ["dark mode", "theme customization", "font size adjustment", "navigation improvements", "simplified layout"],
    "Payment Features": ["new payment methods", "autopay setup", "recurring payments", "bill reminders", "card management"],
    "Authentication": ["fingerprint login", "face recognition", "remember me", "multi-user login", "password recovery"],
    "Language Support": ["Sinhala support", "Tamil support", "multi-language support", "language selection"],
    "Notifications": ["mute notifications", "custom alert tones", "email notifications", "SMS alerts", "notification scheduling"],
    "Data Usage Tracking": ["data balance display", "usage history", "real-time tracking", "data plan management"],
    "Customer Support": ["live chat", "FAQ access", "ticket submission", "call support integration"],
    "Other": ["general feature request", "miscellaneous"]
}

# Helper functions
def is_feature_request(review, threshold=0.8):
    try:
        if not review or pd.isna(review):
            return False
        result = classifier(review, ["feature request", "bug report", "general feedback"])
        return result['labels'][0] == "feature request" and result['scores'][0] >= threshold
    except Exception as e:
        logging.warning(f"Feature request classification failed for review: {review}. Error: {e}")
        return False

def classify_feature_request(review, threshold=0.7):
    try:
        if not review or pd.isna(review):
            return {
                "review": review,
                "main_category": "Other",
                "sub_category": "miscellaneous",
                # "main_confidence": 0.0,
                # "sub_confidence": 0.0
            }
        main_result = classifier(review, main_categories)
        verbose_label = main_result['labels'][0]
        main_score = main_result['scores'][0]
        main_category = category_mapping.get(verbose_label, "Other")

        if main_category == "Other" and main_score < 0.7:
            main_result = classifier(review, main_categories[:-1])
            verbose_label = main_result['labels'][0]
            main_score = main_result['scores'][0]
            main_category = category_mapping.get(verbose_label, "Other")

        sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
        sub_result = classifier(review, sub_list)
        sub_category = sub_result['labels'][0]
        sub_score = sub_result['scores'][0]

        return {
            "review": review,
            "main_category": main_category,
            "sub_category": sub_category,
            # "main_confidence": round(main_score, 2),
            # "sub_confidence": round(sub_score, 2)
        }
    except Exception as e:
        logging.warning(f"Category classification failed for review: {review}. Error: {e}")
        return {
            "review": review,
            "main_category": "Other",
            "sub_category": "miscellaneous",
            # "main_confidence": 0.0,
            # "sub_confidence": 0.0
        }

# Main function
def process_reviews(preprocessed_csv, batch_size=100):
    try:
        # Load preprocessed reviews
        df = pd.read_csv(preprocessed_csv)
        df = df[['userName', 'content', 'translated_content', 'score', 'date', 'source', 'year']].dropna(subset=['translated_content'])
        logging.info(f"Loaded {len(df)} preprocessed reviews")

        feature_reviews = []
        classified_results = []

        # Process in batches
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size]
            logging.info(f"Processing batch {i//batch_size + 1}")

            for _, row in batch.iterrows():
                original = row["content"]
                review = row["translated_content"]
                year = row["year"] if not pd.isna(row["year"]) else "Unknown"
                score = row["score"]
                source = row["source"]

                if is_feature_request(review):
                    feature_reviews.append({
                        "original": original,
                        "translated": review,
                        "score": score,
                        "year": year,
                        "source": source
                    })
                    result = classify_feature_request(review)
                    result.update({
                        # "original": original,
                        # "translated": review,
                        "score": score,
                        "year": year,
                        "source": source
                    })
                    classified_results.append(result)

        feature_df = pd.DataFrame(feature_reviews)
        classified_df = pd.DataFrame(classified_results)

        # Save results
        feature_df.to_csv("feature_requests_only.csv", index=False)
        classified_df.to_csv("categorized_feature_requests.csv", index=False)
        files.download("feature_requests_only.csv")
        files.download("categorized_feature_requests.csv")

        # Generate and save summary
        if not classified_df.empty:
            summary = classified_df.groupby(['main_category', 'sub_category']).size().reset_index(name='count')
            summary.to_csv("feature_requests_summary.csv", index=False)
            files.download("feature_requests_summary.csv")
            logging.info("Summary saved to feature_requests_summary.csv")

        logging.info(f"Processed {len(feature_reviews)} feature requests")
        return feature_df, classified_df

    except Exception as e:
        logging.error(f"Error processing reviews: {e}")
        raise

# Example usage
if __name__ == "__main__":
    try:
        feature_df, classified_df = process_reviews("myslt_preprocessed_reviews.csv")
        print("Processing complete. Files downloaded: feature_requests_only.csv, categorized_feature_requests.csv, feature_requests_summary.csv")
        print("\nSample of categorized feature requests:")
        print(classified_df.head())
    except Exception as e:
        print(f"Failed to process reviews: {e}")

Device set to use cuda:0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing complete. Files downloaded: feature_requests_only.csv, categorized_feature_requests.csv, feature_requests_summary.csv

Sample of categorized feature requests:
                                              review main_category  \
0  The app is improving, please add dark mode and...         UI/UX   
1                               Add a refresh button         UI/UX   
2            need Wifi router password change option         Login   
3  Atleast make a widget for the app to display t...         UI/UX   
4  Need some ui changes, please add circle progre...         UI/UX   

              sub_category  score    year       source  
0                dark mode      3  2024.0  Google Play  
1  navigation improvements      4  2024.0  Google Play  
2        password recovery      1  2024.0  Google Play  
3      theme customization      4  2024.0  Google Play  
4      theme customization      4  2024.0  Google Play  


In [7]:
# !pip install transformers
# !pip install torch


In [8]:
# main_categories = [
#     "User Interface or User Experience issues",
#     "Performance problems",
#     "Network issues",
#     "Billing and Payment problems",
#     "Login or Security issues",
#     "Other"
# ]
# category_mapping = {
#     "User Interface or User Experience issues": "UI/UX",
#     "Performance problems": "Performance",
#     "Network issues": "Network",
#     "Billing and Payment problems": "Billing",
#     "Login or Security issues": "Login/Security",
#     "Other": "Other"
# }


# sub_categories = {
#     "User Interface or User Experience issues": ["dark mode", "navigation issues", "layout problems", "text size", "theme customization"],
#     "Performance problems": ["slow loading", "app crash", "freezing", "high battery usage"],
#    "Network issues": ["no internet", "slow connection", "timeout", "offline mode not working"],
#    "Billing and Payment problems": ["double charge", "bill not updating", "payment failure", "incorrect charges"],
#    "Login or Security issues": ["cannot login", "OTP not received", "session expired", "fingerprint login"],
#   "Other": ["feature request", "language issue", "general feedback"]
# }
# from transformers import pipeline

# # Load zero-shot classifier
# classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# # classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2")

# # classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# def classify_main_and_sub(review):
#     # Step 1: Main category
#     main_result = classifier(review, main_categories)
#     main_category = main_result['labels'][0]

#     # Step 2: Subcategory for the predicted main category
#     sub_list = sub_categories.get(main_category, sub_categories["Other"])
#     sub_result = classifier(review, sub_list)
#     sub_category = sub_result['labels'][0]

#     return {
#         "review": review,
#         "main_category": main_category,
#         "sub_category": sub_category,
#         "confidence": {
#             "main": round(main_result['scores'][0], 2),
#             "sub": round(sub_result['scores'][0], 2)
#         }
#     }


In [9]:
# def classify_main_and_sub(review):
#     main_result = classifier(review, main_categories)
#     verbose_label = main_result['labels'][0]
#     main_category = category_mapping.get(verbose_label, "Other")

#     sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
#     sub_result = classifier(review, sub_list)
#     sub_category = sub_result['labels'][0]

#     return {
#         "review": review,
#         "main_category": main_category,
#         "sub_category": sub_category,
#         "confidence": {
#             "main": round(main_result['scores'][0], 2),
#             "sub": round(sub_result['scores'][0], 2)

#         }
#     }


In [10]:
# def classify_bulk_reviews(reviews):
#     results = []
#     for review in reviews:
#         result = classify_main_and_sub(review)
#         results.append(result)
#     return results
# sample_reviews = [
#     "App crashes every time I try to pay",
#     "Why is there no dark mode?",
#     "Can't login after the update",
#     "Internet not working inside the app"
# ]

# classified = classify_bulk_reviews(sample_reviews)

# # Print the output clearly
# for entry in classified:
#     print(f"\nReview: {entry['review']}")
#     print(f"Main Category: {entry['main_category']} ({entry['confidence']['main']})")
#     print(f"Subcategory: {entry['sub_category']} ({entry['confidence']['sub']})")


Apply Classification to Scraped Reviews (CSV)

In [11]:
# import pandas as pd
# df = pd.read_csv("myslt_reviews.csv")

# # For now, use English-only reviews (translation will come later)
# review_texts = df["content"].dropna().tolist()


In [12]:
# classified_results = classify_bulk_reviews(review_texts)
# classified_df = pd.DataFrame(classified_results)


In [13]:
# classified_df.to_csv("myslt_classified_reviews.csv", index=False)

# # Optional: Download in Colab
# from google.colab import files
# files.download("myslt_classified_reviews.csv")


In [14]:
# from transformers import pipeline
# import pandas as pd
# import ast


# main_categories = [
#     "User Interface or User Experience issues",
#     "Performance problems",
#     "Network issues",
#     "Billing and Payment problems",
#     "Login or Security issues",
#     "Other"
# ]
# # main_categories = [
# #     "Feature request",
# #     "UI/UX enhancement",
# #     "Payment improvements",
# #     "Login enhancements",
# #     "Localization or language support",
# #     "Notification preferences",
# #     "Other"
# # ]

# category_mapping = {
#     "User Interface or User Experience issues": "UI/UX",
#     "Performance problems": "Performance",
#     "Network issues": "Network",
#     "Billing and Payment problems": "Billing",
#     "Login or Security issues": "Login/Security",
#     "Other": "Other"
# }
# # category_mapping = {
# #     "Feature request": "Feature",
# #     "UI/UX enhancement": "UI/UX",
# #     "Payment improvements": "Payment",
# #     "Login enhancements": "Login",
# #     "Localization or language support": "Language",
# #     "Notification preferences": "Notifications",
# #     "Other": "Other"
# # }

# # Expanded subcategories under "Other"
# sub_categories = {
#     "User Interface or User Experience issues": ["dark mode", "navigation issues", "layout problems", "text size", "theme customization"],
#     "Performance problems": ["slow loading", "app crash", "freezing", "high battery usage"],
#     "Network issues": ["no internet", "slow connection", "timeout", "offline mode not working"],
#     "Billing and Payment problems": ["double charge", "bill not updating", "payment failure", "incorrect charges"],
#     "Login or Security issues": ["cannot login", "OTP not received", "session expired", "fingerprint login"],
#     "Other": ["feature request", "language issue", "unwanted notifications", "app update problem", "advertisement complaint", "general feedback"]
# }
# sub_categories = {
#     "Feature request": ["dark mode", "fingerprint login", "multiple account support", "bill reminder", "offline access"],
#     "UI/UX enhancement": ["theme customization", "font size control", "layout feedback", "navigation improvements"],
#     "Payment improvements": ["more payment methods", "autopay setup", "bill history filter", "add card feature"],
#     "Login enhancements": ["biometric login", "remember me", "faster login", "multi-user login"],
#     "Localization or language support": ["Sinhala support", "Tamil support", "multi-language support"],
#     "Notification preferences": ["mute SMS alerts", "email-only notifications", "custom alert tones"],
#     "Other": ["unclassified request", "feature idea"]
# }


# classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# def classify_main_and_sub(review, confidence_threshold=0.6):
#     # Step 1: Predict main category
#     main_result = classifier(review, main_categories)
#     verbose_label = main_result['labels'][0]
#     main_score = main_result['scores'][0]

#     main_category = category_mapping.get(verbose_label, "Other")

#     # Step 2: Predict subcategory
#     sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
#     sub_result = classifier(review, sub_list)
#     sub_category = sub_result['labels'][0]
#     sub_score = sub_result['scores'][0]

#     # Optionally flag low-confidence predictions
#     flagged = main_score < confidence_threshold or sub_score < confidence_threshold

#     return {
#         "review": review,
#         "main_category": main_category,
#         "sub_category": sub_category,
#         "main_confidence": round(main_score, 2),
#         "sub_confidence": round(sub_score, 2),
#         "low_confidence_flag": flagged
#     }

# # --------------------------
# def classify_bulk_reviews(reviews):
#     results = []
#     for review in reviews:
#         results.append(classify_main_and_sub(review))
#     return results


# df = pd.read_csv("myslt_reviews.csv")
# review_texts = df["content"].dropna().tolist()

# classified_results = classify_bulk_reviews(review_texts)
# classified_df = pd.DataFrame(classified_results)


# classified_df.to_csv("myslt_classified_reviews.csv", index=False)

# # For Colab downloads (if needed)
# from google.colab import files
# files.download("myslt_classified_reviews.csv")


In [15]:
# from transformers import pipeline
# import pandas as pd
# from datetime import datetime

# # -----------------------------------------------
# # 1. Load Classifier (Multilingual Zero-Shot Model)
# # -----------------------------------------------
# classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# # -----------------------------------------------
# # 2. Developer-Useful Categories
# # -----------------------------------------------
# main_categories = [
#     "UI/UX enhancement",
#     "Payment improvements",
#     "Login enhancements",
#     "Localization or language support",
#     "Notification preferences",
#     "Other"
# ]

# category_mapping = {
#     "UI/UX enhancement": "UI/UX",
#     "Payment improvements": "Payment",
#     "Login enhancements": "Login",
#     "Localization or language support": "Language",
#     "Notification preferences": "Notifications",
#     "Other": "Other"
# }

# sub_categories = {
#     "UI/UX enhancement": [
#         "dark mode", "theme customization", "font size control", "navigation improvements", "layout feedback"
#     ],
#     "Payment improvements": [
#         "add new payment method", "autopay setup", "recurring payments", "bill reminders", "add card feature"
#     ],
#     "Login enhancements": [
#         "fingerprint login", "face recognition", "remember me", "multi-user login"
#     ],
#     "Localization or language support": [
#         "Sinhala support", "Tamil support", "multi-language", "language selection"
#     ],
#     "Notification preferences": [
#         "mute alerts", "custom alert tone", "email notifications", "SMS alerts", "notification scheduling"
#     ],
#     "Other": ["unclassified feature", "general feature idea"]
# }

# # -----------------------------------------------
# # 3. Step 1: Detect Feature Request or Not
# # -----------------------------------------------
# def is_feature_request(review, threshold=0.7):
#     result = classifier(review, ["feature request", "bug report", "general feedback"])
#     return result['labels'][0] == "feature request" and result['scores'][0] >= threshold

# # -----------------------------------------------
# # 4. Step 2: Classify Feature Requests
# # -----------------------------------------------
# def classify_feature_request(review, threshold=0.6):
#     main_result = classifier(review, main_categories)
#     verbose_label = main_result['labels'][0]
#     main_score = main_result['scores'][0]
#     main_category = category_mapping.get(verbose_label, "Other")

#     sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
#     sub_result = classifier(review, sub_list)
#     sub_category = sub_result['labels'][0]
#     sub_score = sub_result['scores'][0]

#     return {
#         "review": review,
#         "main_category": main_category,
#         "sub_category": sub_category,
#         "main_confidence": round(main_score, 2),
#         "sub_confidence": round(sub_score, 2)
#     }

# # -----------------------------------------------
# # 5. Step 3: Process All Reviews
# # -----------------------------------------------
# def process_reviews(file_path):
#     df = pd.read_csv(file_path)
#     df = df.dropna(subset=["content"])

#     # Convert 'date' column to datetime and extract year
#     df["at"] = pd.to_datetime(df["at"], errors='coerce') # Changed "date" to "at"
#     df["year"] = df["at"].dt.year # Changed "date" to "at"

#     feature_reviews = []
#     classified_results = []

#     for _, row in df.iterrows():
#         review = row["content"]
#         year = row["year"]

#         if is_feature_request(review):
#             feature_reviews.append({"review": review, "year": year})
#             result = classify_feature_request(review)
#             result["year"] = year
#             classified_results.append(result)

#     # Save all feature requests to CSV (Step 1)
#     feature_df = pd.DataFrame(feature_reviews)
#     feature_df.to_csv("feature_requests_only.csv", index=False)

#     # Save categorized feature requests to CSV (Step 3)
#     classified_df = pd.DataFrame(classified_results)
#     classified_df.to_csv("categorized_feature_requests.csv", index=False)

#     print(" Feature requests extracted and classified.")
#     return classified_df

# # -----------------------------------------------
# # 6. Run the Pipeline
# # -----------------------------------------------
# classified_df = process_reviews("myslt_reviews.csv")

In [16]:
# from google.colab import files
# files.download("feature_requests_only.csv")
# files.download("categorized_feature_requests.csv")


In [17]:
# !pip install transformers langdetect googletrans==4.0.0-rc1

# from transformers import pipeline
# import pandas as pd
# from datetime import datetime
# from langdetect import detect
# from googletrans import Translator

# # Load models
# classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")
# translator = Translator()

# # Developer-friendly category structure
# main_categories = [
#     "UI/UX enhancement",
#     "Payment improvements",
#     "Login enhancements",
#     "Localization or language support",
#     "Notification preferences",
#     "Other"
# ]

# category_mapping = {
#     "UI/UX enhancement": "UI/UX",
#     "Payment improvements": "Payment",
#     "Login enhancements": "Login",
#     "Localization or language support": "Language",
#     "Notification preferences": "Notifications",
#     "Other": "Other"
# }

# sub_categories = {
#     "UI/UX enhancement": ["dark mode", "theme customization", "font size control", "navigation improvements", "layout feedback"],
#     "Payment improvements": ["add new payment method", "autopay setup", "recurring payments", "bill reminders", "add card feature"],
#     "Login enhancements": ["fingerprint login", "face recognition", "remember me", "multi-user login"],
#     "Localization or language support": ["Sinhala support", "Tamil support", "multi-language", "language selection"],
#     "Notification preferences": ["mute alerts", "custom alert tone", "email notifications", "SMS alerts", "notification scheduling"],
#     "Other": ["unclassified feature", "general feature idea"]
# }

# # Translation and classification helpers
# def is_feature_request(review, threshold=0.7):
#     result = classifier(review, ["feature request", "bug report", "general feedback"])
#     return result['labels'][0] == "feature request" and result['scores'][0] >= threshold

# def classify_feature_request(review, threshold=0.6):
#     main_result = classifier(review, main_categories)
#     verbose_label = main_result['labels'][0]
#     main_score = main_result['scores'][0]
#     main_category = category_mapping.get(verbose_label, "Other")

#     sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
#     sub_result = classifier(review, sub_list)
#     sub_category = sub_result['labels'][0]
#     sub_score = sub_result['scores'][0]

#     return {
#         "review": review,
#         "main_category": main_category,
#         "sub_category": sub_category,
#         "main_confidence": round(main_score, 2),
#         "sub_confidence": round(sub_score, 2)
#     }

# def translate_review(text):
#     try:
#         if detect(text) != 'en':
#             return translator.translate(text, dest='en').text
#         else:
#             return text
#     except:
#         return text

# # Main function
# def process_reviews(file_path):
#     df = pd.read_csv(file_path)
#     df = df.dropna(subset=["content"])

#     df["translated_content"] = df["content"].apply(translate_review)
#     df["date"] = pd.to_datetime(df["date"], errors='coerce')
#     df["year"] = df["date"].dt.year

#     feature_reviews = []
#     classified_results = []

#     for _, row in df.iterrows():
#         original = row["content"]
#         review = row["translated_content"]
#         year = row["year"]

#         if is_feature_request(review):
#             feature_reviews.append({"original": original, "translated": review, "year": year})
#             result = classify_feature_request(review)
#             result.update({"original": original, "translated": review, "year": year})
#             classified_results.append(result)

#     feature_df = pd.DataFrame(feature_reviews)
#     classified_df = pd.DataFrame(classified_results)

#     feature_df.to_csv("feature_requests_only.csv", index=False)
#     classified_df.to_csv("categorized_feature_requests.csv", index=False)

#     return feature_df, classified_df

# # Example usage (after uploading CSV file)
# # feature_df, classified_df = process_reviews("myslt_reviews.csv")


In [18]:
# from google.colab import files
# files.download("feature_requests_only.csv")
# files.download("categorized_feature_requests.csv")


# update within 24hrs

In [19]:
#Required installations
# Run these once in your environment (e.g., Colab or local)
!pip install google-play-scraper deep-translator transformers pandas --quiet

# Full Pipeline: Scrape, Translate, Classify, and Save
from google_play_scraper import reviews_all
from transformers import pipeline
from deep_translator import GoogleTranslator
import pandas as pd
from datetime import datetime, timedelta
import os

# Zero-Shot Classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Main categories
main_categories = [
    "User Interface or User Experience issues",
    "Performance problems",
    "Network issues",
    "Billing and Payment problems",
    "Login or Security issues",
    "Other"
]

category_mapping = {
    "User Interface or User Experience issues": "UI/UX",
    "Performance problems": "Performance",
    "Network issues": "Network",
    "Billing and Payment problems": "Billing",
    "Login or Security issues": "Login/Security",
    "Other": "Other"
}

# Subcategories
sub_categories = {
    "User Interface or User Experience issues": ["dark mode", "navigation issues", "layout problems", "text size", "theme customization"],
    "Performance problems": ["slow loading", "app crash", "freezing", "high battery usage"],
    "Network issues": ["no internet", "slow connection", "timeout", "offline mode not working"],
    "Billing and Payment problems": ["double charge", "bill not updating", "payment failure", "incorrect charges"],
    "Login or Security issues": ["cannot login", "OTP not received", "session expired", "fingerprint login"],
    "Other": ["feature request", "language issue", "general feedback"]
}

# Translate if needed
def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return text

# Classification function
def classify_main_and_sub(review):
    main_result = classifier(review, main_categories)
    verbose_label = main_result['labels'][0]
    main_category = category_mapping.get(verbose_label, "Other")

    sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
    sub_result = classifier(review, sub_list)
    sub_category = sub_result['labels'][0]

    return {
        "review": review,
        "main_category": main_category,
        "sub_category": sub_category,
        "confidence_main": round(main_result['scores'][0], 2),
        "confidence_sub": round(sub_result['scores'][0], 2)
    }

# Fetch Google Play reviews from last 24 hours
def fetch_new_reviews_last_24hrs():
    all_reviews = reviews_all(
        'com.slt.selfcare',
        lang='en',
        country='lk',
        sleep_milliseconds=1000
    )
    now = datetime.utcnow()
    yesterday = now - timedelta(hours=24)

    reviews_24hr = []
    for review in all_reviews:
        posted_date = review.get('at')
        if posted_date and posted_date >= yesterday:
            reviews_24hr.append(review)

    return reviews_24hr

#  Update the CSV file with new reviews
def update_review_classification_24hr():
    reviews = fetch_new_reviews_last_24hrs()
    if not reviews:
        print("No new reviews in last 24 hours.")
        return

    print(f"Found {len(reviews)} new reviews from last 24 hours...")

    # Load existing if exists
    if os.path.exists("myslt_classified_reviews.csv"):
        df_existing = pd.read_csv("myslt_classified_reviews.csv")
        existing_texts = set(df_existing["review"].dropna())
    else:
        df_existing = pd.DataFrame()
        existing_texts = set()

    new_entries = []
    for rev in reviews:
        raw_text = rev.get("content", "")
        if raw_text and raw_text not in existing_texts:
            eng_text = translate_to_english(raw_text)
            result = classify_main_and_sub(eng_text)
            result["original"] = raw_text
            result["translated"] = eng_text
            result["timestamp"] = rev["at"]
            new_entries.append(result)

    if new_entries:
        df_new = pd.DataFrame(new_entries)
        df_final = pd.concat([df_existing, df_new], ignore_index=True)
        df_final.to_csv("myslt_classified_reviews.csv", index=False)
        print(f"Updated myslt_classified_reviews.csv with {len(new_entries)} new classified reviews.")
    else:
        print("No truly new reviews to classify.")

# Run once per day manually or with scheduler
if __name__ == "__main__":
    update_review_classification_24hr()


Device set to use cuda:0


No new reviews in last 24 hours.


# Summarize Feature Requests

In [20]:
import pandas as pd

# Load classified results
df = pd.read_csv("myslt_classified_reviews.csv")

# Group by main and subcategory
summary = df.groupby(["main_category", "sub_category"]).size().reset_index(name="count")

# Create plain text summary for email
report_lines = ["📌 Weekly Feature Request Summary:\n"]
for _, row in summary.iterrows():
    line = f"- {row['main_category']} > {row['sub_category']}: {row['count']} requests"
    report_lines.append(line)

email_body = "\n".join(report_lines)
print(email_body)  # Preview


FileNotFoundError: [Errno 2] No such file or directory: 'myslt_classified_reviews.csv'

# Send Email

In [27]:
import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart # Import MIMEMultipart
import pandas as pd

def send_email_with_attachment(subject, body, to_email, from_email, smtp_server, smtp_port, login, password, attachment_path=None):
    # Create a multipart message
    msg = MIMEMultipart()
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = to_email

    # Attach the body of the email
    msg.attach(MIMEText(body, 'plain'))

    if attachment_path:
        try:
            with open(attachment_path, "rb") as f:
                attach = MIMEApplication(f.read(),_subtype="csv") # Specify subtype as csv
            attach.add_header('Content-Disposition', 'attachment', filename=attachment_path)
            msg.attach(attach)
        except Exception as e:
            print(f"Error attaching file {attachment_path}: {e}")
            # Decide how to handle attachment failure - here we print error and continue
            # You might want to return or raise the exception depending on desired behavior
            pass


    try:
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            server.login(login, password)
            server.sendmail(from_email, to_email, msg.as_string())
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print("SMTP Authentication Error: Make sure your username and password are correct.")
        print("If using Gmail, ensure Less Secure App access is enabled or use an App Password if 2FA is on.")
    except Exception as e:
        print(f"An error occurred while sending email: {e}")

# Load the categorized feature requests from the CSV file
try:
    # Check if the file exists before trying to attach it
    import os
    if os.path.exists("categorized_feature_requests.csv"):
        email_body = "Please find the categorized feature requests attached." # Simple body for email
        attachment_file = "categorized_feature_requests.csv"
    else:
        email_body = "The categorized feature requests CSV file was not found."
        attachment_file = None

except Exception as e:
    email_body = f"An error occurred while preparing the email: {e}"
    attachment_file = None


sender_email =  "kaveeshanirmani25@gmail.com"

app_password = "cidkljqictlwmady"
send_email_with_attachment(
    subject="MYSLT Categorized Feature Requests",
    body=email_body,
    to_email="yehanmanodya12@gmail.com",
    from_email=sender_email,
    smtp_server="smtp.gmail.com",
    smtp_port=465, # Use port 465 for SSL
    login=sender_email,
    password=app_password, # Pass the App Password here
    attachment_path=attachment_file # Pass the attachment file path
)

Email sent successfully!
